In [1]:
import pandas as pd
import numpy as np
import torch
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data_to_import.csv')

In [3]:
df

,id,url,text,w_what,w_where,w_when,w_who_author,w_who_victim,w_who_other,w_why,h_how,newspaper,activelearning_tag
0,200,https://gazzettadimodena.gelocal.it/modena/cro...,SERRAMAZZONI. Fuoristrada rubato trovato e res...,"[""fuoristrada""]","[""SERRAMAZZONI""]","[""Luned\u00ec""]",[],"[""titolare""]",[],NaN,NaN,Gazzetta di Modena,furto
1,209,https://gazzettadimodena.gelocal.it/modena/cro...,"Tre anni, un mese e dieci giorni di carcere è ...","[""5 euro""]","[""via Buozzi""]","[""mattina del 20 marzo 2019""]","[""modenese nomade di etnia sinti di 41 anni""]","[""una donna di 89 anni""]","[""carabinieri""]",NaN,NaN,Gazzetta di Modena,furto
2,210,https://gazzettadimodena.gelocal.it/modena/cro...,Un furto in appartamento è stato messo a segno...,"[""gioielli e preziosi"", ""cassaforte""]","[""via Santa Caterina, zona Crocetta""]","[""nella serata di ieri""]","[""ladri""]","[""appartamento"", ""proprietari""]","[""la Polizia""]",NaN,NaN,Gazzetta di Modena,furto
3,217,https://gazzettadimodena.gelocal.it/modena/cro...,"MODENA. Condannata a 3 anni, 1 mese e 10 giorn...","[""5 euro""]","[""viale Gramsci""]",[],"[""donna di origine sinti e residente a Modena""]","[""signora""]","["" ""]",NaN,NaN,Gazzetta di Modena,furto
4,219,https://gazzettadimodena.gelocal.it/modena/cro...,MODENA «È la prima volta che subiamo un furto ...,"[""cappotti e abiti femminili in esposizione"", ...","[""via del Taglio""]","[""nottetempo""]","[""ladro solo o in coppia""]","[""La Sacoche, il negozio di abbigliamento femm...","[""polizia""]",NaN,NaN,Gazzetta di Modena,furto
...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,2953227,https://gazzettadimodena.gelocal.it/modena/cro...,"MODENA «Beh, alla fine sono stati ladri capaci...","[""computer Mac""]","[""Alla carrozzeria Ferraguti e Cavani"", ""in vi...","[""luned\u00ec sera verso alle 21.40""]","[""ladri""]","[""Maurizio Muzzioli""]","[""quelli dell\u2019antifurto""]",NaN,NaN,Gazzetta di Modena,furto
281,3063703,https://gazzettadimodena.gelocal.it/modena/cro...,FINALE Ora la vettura è appoggiata su quattro ...,"[""i pochi soldi abbandonati in cassa""]","[""FINALE"", ""in varie attivit\u00e0 commerciali...","[""da alcuni mesi""]","[""Yuri Sabattini e Alessandra Mengozzi""]",[],"[""carabinieri""]",NaN,NaN,Gazzetta di Modena,furto
282,3194911,https://gazzettadimodena.gelocal.it/modena/cro...,"CASTELFRANCO. Ieri notte, i carabinieri della ...",[],"[""in appartamento""]","[""Ieri notte""]","[""un cittadino rumeno, classe 1999"", ""un compl...",[],"[""i carabinieri della Compagnia di Modena"", ""R...",NaN,NaN,Gazzetta di Modena,furto
283,3236698,https://gazzettadimodena.gelocal.it/modena/cro...,"Daniele Montanari «Bene le telecamere, ma risp...","[""il 16 novembre""]","[""ai Torrazzi""]","[""il 16 novembre""]","[""professionisti"", ""un uomo""]","[""la Pneustorrazz"", ""cotitolare del bar Moka""]",[],NaN,NaN,Gazzetta di Modena,furto


In [7]:
from neo4j import GraphDatabase

class Neo4jDb:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        
    def close(self):
        self.driver.close()
        
    def write_data(self):
        with self.driver.session() as session:
            session.run("CALL n10s.graphconfig.init({handleVocabUris: 'MAP', handleMultival: 'ARRAY', keepLangTag: false, keepCustomDataTypes: true, applyNeo4jNaming: true })")
            session.write_transaction(self.run_transaction)
            
    @staticmethod
    def run_transaction(tx):
        tx.run("""LOAD CSV WITH HEADERS FROM \"file:///data_to_import.csv\" AS line FIELDTERMINATOR \",\" 
                MERGE(c:CrimeNews {id: line.id})
                SET c += {name: line.id, url: line.url, text: line.text}
                WITH line, c
                UNWIND split(replace(replace(line.w_what, "[", ""), "]", ""), '\", \"') AS w_what 
                WITH DISTINCT w_what, c
                where size(toLower(trim(replace(w_what, '\"', ""))))>1
                MERGE (what:What {name: toLower(trim(replace(w_what, '\"', "")))})
                MERGE (c)-[r:WHAT]->(what)""")
        
        tx.run("""LOAD CSV WITH HEADERS FROM \"file:///data_to_import.csv\" AS line FIELDTERMINATOR \",\" 
                MERGE(c:CrimeNews {id: line.id})
                WITH line, c
                UNWIND split(replace(replace(line.w_where, "[",""), "]", ""), '\", \"') AS w_where 
                WITH DISTINCT w_where, c
                where size(toLower(trim(replace(w_where, '\"', ""))))>1
                MERGE (where:Where {name: toLower(trim(replace(w_where, '\"', "")))})
                MERGE (c)-[r:WHERE]->(where)""")
        
        tx.run("""LOAD CSV WITH HEADERS FROM \"file:///data_to_import.csv\" AS line FIELDTERMINATOR \",\" 
                MERGE(c:CrimeNews {id: line.id})
                WITH line, c
                UNWIND split(replace(replace(line.w_when, "[",""), "]", ""), '\", \"') AS w_when 
                WITH DISTINCT w_when, c
                where size(toLower(trim(replace(w_when, '\"', ""))))>1
                MERGE (when:When {name: toLower(trim(replace(w_when, '\"', "")))})
                MERGE (c)-[r:WHEN]->(when)""")
        
        tx.run("""LOAD CSV WITH HEADERS FROM \"file:///data_to_import.csv\" AS line FIELDTERMINATOR \",\" 
                MERGE(c:CrimeNews {id: line.id})
                WITH line, c
                UNWIND split(replace(replace(line.w_who_author, "[",""), "]", ""), '\", \"') AS w_who_author 
                WITH DISTINCT w_who_author, c
                where size(toLower(trim(replace(w_who_author, '\"', ""))))>1
                MERGE (who_author:Who_author {name: toLower(trim(replace(w_who_author, '\"', "")))})
                MERGE (c)-[r:WHO_AUTHOR]->(who_author)""")
        
        tx.run("""LOAD CSV WITH HEADERS FROM \"file:///data_to_import.csv\" AS line FIELDTERMINATOR \",\" 
                MERGE(c:CrimeNews {id: line.id})
                WITH line, c
                UNWIND split(replace(replace(line.w_who_victim, "[",""), "]", ""), '\", \"') AS w_who_victim 
                WITH DISTINCT w_who_victim, c
                where size(toLower(trim(replace(w_who_victim, '\"', ""))))>1
                MERGE (who_victim:Who_victim {name: toLower(trim(replace(w_who_victim, '\"', "")))})
                MERGE (c)-[r:WHO_VICTIM]->(who_victim)""")
        
        tx.run("""LOAD CSV WITH HEADERS FROM \"file:///data_to_import.csv\" AS line FIELDTERMINATOR \",\" 
                MERGE(c:CrimeNews {id: line.id})
                WITH line, c
                UNWIND split(replace(replace(line.w_who_other, "[",""), "]", ""), '\", \"') AS w_who_other 
                WITH DISTINCT w_who_other, c
                where size(toLower(trim(replace(w_who_other, '\"', ""))))>1
                MERGE (who_other:Who_other {name: toLower(trim(replace(w_who_other, '\"', "")))})
                MERGE (c)-[r:WHO_OTHER]->(who_other)""")
        
    def update_data(self):
        with self.driver.session() as session:
            session.run("CALL n10s.graphconfig.init({handleVocabUris: 'MAP', handleMultival: 'ARRAY', keepLangTag: false, keepCustomDataTypes: true, applyNeo4jNaming: true })")
            session.write_transaction(self.add_similarities)
            
    @staticmethod
    def add_similarities(tx):
        Ws=['what', 'when', 'who_author', 'who_victim', 'who_other']
        for w in Ws:
            tx.run("""LOAD CSV WITH HEADERS FROM \"file:///"""+w+"""_similarities.csv\" AS line FIELDTERMINATOR \";\" 
                    MERGE(w1:"""+w.capitalize()+""" {name: line.text1})
                    with w1, line
                    MERGE(w2:"""+w.capitalize()+""" {name: line.text2})
                    with w1, w2, line
                    MERGE (w1)-[r:SIM_"""+w.upper()+"""]-(w2)
                    SET r.weight=line.distance""")
       
if __name__ == '__main__':
    db = Neo4jDb("bolt://localhost:7687", "neo4j", "neo4j_password")
    db.write_data()
    db.close()

In [8]:
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import torch
name='dbmdz/bert-base-italian-xxl-cased'
model = BertModel.from_pretrained(name, output_hidden_states = True)
tokenizer = BertTokenizer.from_pretrained(name)

In [9]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32102, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [10]:
def bert_text_preparation(text, tokenizer):
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensors

In [11]:
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2][1:]

    token_embeddings = hidden_states[-1]
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

    return list_token_embeddings

In [12]:
from scipy.spatial.distance import cosine

In [13]:
Ws=['what', 'when', 'who_author', 'who_victim', 'who_other']

for w in Ws:
    print(w)
    
    my_w=set()
    for index, row in df.iterrows():
        print("-----------------------")
        print(row['w_'+w])
        elements=row['w_'+w].split("\", \"")
        for el in elements:
            el=el.replace("\"","").replace("[","").replace("]","")
            el=el.strip()
            print(el)
            if(len(el)>1):
                my_w.add(el.lower())
    print(my_w)
    
    target_word_embeddings = []    
    for text in my_w:
        tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
        list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
        column_average = [sum(sub_list) / len(sub_list) for sub_list in zip(*list_token_embeddings)]
        target_word_embeddings.append(column_average)
        
    list_of_distances = []
    for text1, embed1 in zip(my_w, target_word_embeddings):
        for text2, embed2 in zip(my_w, target_word_embeddings):
            if(text1>text2):
                cos_dist = 1 - cosine(embed1, embed2)
                list_of_distances.append([text1, text2, cos_dist])
    distances_df = pd.DataFrame(list_of_distances, columns=['text1', 'text2', 'distance'])
    
    highest_sim=distances_df[(distances_df.distance>=0.85) & (distances_df.distance!=1)]
    
    highest_sim.to_csv(w+"_similarities.csv", index=False, sep=';')

what
-----------------------
["fuoristrada"]
fuoristrada
-----------------------
["5 euro"]
5 euro
-----------------------
["gioielli e preziosi", "cassaforte"]
gioielli e preziosi
cassaforte
-----------------------
["5 euro"]
5 euro
-----------------------
["cappotti e abiti femminili in esposizione", "400 euro in contanti"]
cappotti e abiti femminili in esposizione
400 euro in contanti
-----------------------
["preziosi"]
preziosi
-----------------------
["oro e contanti"]
oro e contanti
-----------------------
["una borsetta"]
una borsetta
-----------------------
["materiale"]
materiale
-----------------------
["prodotti tra i pi\u00f9 costosi, per un valore di decine e decine di migliaia d\u2019euro"]
prodotti tra i pi\u00f9 costosi, per un valore di decine e decine di migliaia d\u2019euro
-----------------------
["un\u2019Alfa Romeo \u201cGiulietta\u201d"]
un\u2019Alfa Romeo \u201cGiulietta\u201d
-----------------------
["borsa con documenti, telefono, carte e soldi"]
borsa con do

["chiavi inglesi, trapani, apparecchiature elettroniche per diagnostica di auto, un ciclomotore per un valore pari a circa 5mila euro"]
chiavi inglesi, trapani, apparecchiature elettroniche per diagnostica di auto, un ciclomotore per un valore pari a circa 5mila euro
-----------------------
["13 mila euro tra danni a vetrine e abiti da sposa trafugati"]
13 mila euro tra danni a vetrine e abiti da sposa trafugati
-----------------------
["denaro"]
denaro
-----------------------
["borsetta"]
borsetta
-----------------------
["cellulare e il portafoglio"]
cellulare e il portafoglio
-----------------------
["telefono cellulare"]
telefono cellulare
-----------------------
["una serie di utensili", "del valore di oltre 3mila euro"]
una serie di utensili
del valore di oltre 3mila euro
-----------------------
[" borsetta", "portafoglio con qualche soldo e soprattutto con tutti i documenti", "borsa", "cellulare "]
borsetta
portafoglio con qualche soldo e soprattutto con tutti i documenti
borsa


-----------------------
["105 euro"]
105 euro
-----------------------
["Un portafogli"]
Un portafogli
-----------------------
["slot dentro le quali c\u2019erano circa 4mila euro"]
slot dentro le quali c\u2019erano circa 4mila euro
-----------------------
["pochette"]
pochette
-----------------------
["25mila euro di gomme"]
25mila euro di gomme
-----------------------
[]

-----------------------
["affettatrice"]
affettatrice
-----------------------
["l\u2019altra sera"]
l\u2019altra sera
-----------------------
["borsetta"]
borsetta
-----------------------
["Kawasaki 650"]
Kawasaki 650
-----------------------
["La bicicletta, una MTB del valore commerciale di circa 400 euro"]
La bicicletta, una MTB del valore commerciale di circa 400 euro
-----------------------
["un computer"]
un computer
-----------------------
["calzature per un ammontare superiore a 1200 euro"]
calzature per un ammontare superiore a 1200 euro
-----------------------
["le componenti anteriori", "fari al paraurti", 

when
-----------------------
["Luned\u00ec"]
Luned\u00ec
-----------------------
["mattina del 20 marzo 2019"]
mattina del 20 marzo 2019
-----------------------
["nella serata di ieri"]
nella serata di ieri
-----------------------
[]

-----------------------
["nottetempo"]
nottetempo
-----------------------
["luglio del 2018"]
luglio del 2018
-----------------------
["sabato sera", "tra le 18.30 e le 19.30"]
sabato sera
tra le 18.30 e le 19.30
-----------------------
["met\u00e0 di febbraio"]
met\u00e0 di febbraio
-----------------------
["di notte"]
di notte
-----------------------
["nella notte tra domenica e luned\u00ec"]
nella notte tra domenica e luned\u00ec
-----------------------
["attorno alle 22"]
attorno alle 22
-----------------------
["serata di marted\u00ec"]
serata di marted\u00ec
-----------------------
["nella serata di luned\u00ec", "alle 21.20"]
nella serata di luned\u00ec
alle 21.20
-----------------------
[" il 30 ottobre"]
il 30 ottobre
-----------------------
[]



-----------------------
[]

-----------------------
["3 e 4 agosto"]
3 e 4 agosto
-----------------------
[]

-----------------------
[]

-----------------------
[]

-----------------------
["marted\u00ec notte"]
marted\u00ec notte
-----------------------
["Il 29 gennaio, nel pomeriggio"]
Il 29 gennaio, nel pomeriggio
-----------------------
["la sera di sabato 26 settembre"]
la sera di sabato 26 settembre
-----------------------
["sabato"]
sabato
-----------------------
["sabato"]
sabato
-----------------------
[]

-----------------------
[]

-----------------------
["nella notte tra il 18 e il 19 novembre"]
nella notte tra il 18 e il 19 novembre
-----------------------
["nel cuore dell anotte"]
nel cuore dell anotte
-----------------------
["Verso le 20 di mercoled\u00ec sera"]
Verso le 20 di mercoled\u00ec sera
-----------------------
["il 26 settembre"]
il 26 settembre
-----------------------
["Intorno alle 12 di mercoled\u00ec"]
Intorno alle 12 di mercoled\u00ec
------------------

...

["il titolare di una concessionaria di auto multimarche"]
il titolare di una concessionaria di auto multimarche
-----------------------
["ladri", "tre: un palo e due in azione"]
ladri
tre: un palo e due in azione
-----------------------
["Un 30enne pakistano"]
Un 30enne pakistano
-----------------------
["un 27enne residente nel Reggiano"]
un 27enne residente nel Reggiano
-----------------------
["Un uomo di origini tunisine di 29 anni"]
Un uomo di origini tunisine di 29 anni
-----------------------
[]

-----------------------
["J.M. di 17 anni e nazionalit\u00e0 tunisina"]
J.M. di 17 anni e nazionalit\u00e0 tunisina
-----------------------
["ladri", "ladri"]
ladri
ladri
-----------------------
["due giovani maghrebini"]
due giovani maghrebini
-----------------------
["48enne", "modenese che vive in citt\u00e0"]
48enne
modenese che vive in citt\u00e0
-----------------------
["malviventi"]
malviventi
-----------------------
["una banda di ladri", "tre persone"]
una banda di ladri
tre pe

who_victim
-----------------------
["titolare"]
titolare
-----------------------
["una donna di 89 anni"]
una donna di 89 anni
-----------------------
["appartamento", "proprietari"]
appartamento
proprietari
-----------------------
["signora"]
signora
-----------------------
["La Sacoche, il negozio di abbigliamento femminile"]
La Sacoche, il negozio di abbigliamento femminile
-----------------------
["40enne brasiliana"]
40enne brasiliana
-----------------------
["tre appartamenti", "un residence"]
tre appartamenti
un residence
-----------------------
[]

-----------------------
["un 83enne"]
un 83enne
-----------------------
["La Scam"]
La Scam
-----------------------
["il proprietario della vettura rubata, un 40 enne di Crespellano", "proprietari"]
il proprietario della vettura rubata, un 40 enne di Crespellano
proprietari
-----------------------
["donna"]
donna
-----------------------
["Tred"]
Tred
-----------------------
["chiesa di magreta"]
chiesa di magreta
--------------------

-----------------------
[]

-----------------------
[]

-----------------------
["Stefania Ascari, direttrice del Distretto sanitario di Carpi, e Andrea Ziglio, direttore medico del Ramazzini"]
Stefania Ascari, direttrice del Distretto sanitario di Carpi, e Andrea Ziglio, direttore medico del Ramazzini
-----------------------
["Temporary Outlet", "negozio di abbigliamento", "titolari"]
Temporary Outlet
negozio di abbigliamento
titolari
-----------------------
[]

-----------------------
["diciottenne"]
diciottenne
-----------------------
["SAN FELICE", "una donna di Modena"]
SAN FELICE
una donna di Modena
-----------------------
["58enne proprietario "]
58enne proprietario
-----------------------
["emporio solidale \u201cIl Melograno\u201d"]
emporio solidale \u201cIl Melograno\u201d
-----------------------
["Federica Carretti"]
Federica Carretti
-----------------------
["bar", "centro scommesse", "Zenzero Caf\u00e8", "locale", "Eurobet"]
bar
centro scommesse
Zenzero Caf\u00e8
locale
Eu

who_other
-----------------------
[]

-----------------------
["carabinieri"]
carabinieri
-----------------------
["la Polizia"]
la Polizia
-----------------------
[" "]

-----------------------
["polizia"]
polizia
-----------------------
["avvocato Davide Ascari", "un amico della escort"]
avvocato Davide Ascari
un amico della escort
-----------------------
["inquilini e proprietari"]
inquilini e proprietari
-----------------------
["Carabinieri della stazione sassolese"]
Carabinieri della stazione sassolese
-----------------------
["cane", "la vigilanza", "i carabinieri"]
cane
la vigilanza
i carabinieri
-----------------------
["polizia"]
polizia
-----------------------
[]

-----------------------
["personale"]
personale
-----------------------
[]

-----------------------
["carabinieri"]
carabinieri
-----------------------
[]

-----------------------
["la polizia", "La pattuglia della Squadra volante"]
la polizia
La pattuglia della Squadra volante
-----------------------
["guardia giu

i Carabinieri di Carpi
-----------------------
["agenti"]
agenti
-----------------------
["I carabinieri della sezione Radiomobile della compagnia di Modena"]
I carabinieri della sezione Radiomobile della compagnia di Modena
-----------------------
["la Polizia di Carpi "]
la Polizia di Carpi
-----------------------
["I Carabinieri di Spilamberto", "colleghi di Poggio di Ancona"]
I Carabinieri di Spilamberto
colleghi di Poggio di Ancona
-----------------------
["carabinieri di San Felice"]
carabinieri di San Felice
-----------------------
[]

-----------------------
["un amico di Gherardini"]
un amico di Gherardini
-----------------------
["benzinaio l\u00ec di fianco", "la polizia"]
benzinaio l\u00ec di fianco
la polizia
-----------------------
["carabinieri"]
carabinieri
-----------------------
["tecnici di \u201cEnele distribuzione\u201d", "i Carabinieri del Comando Stazione di Fiorano Modenese"]
tecnici di \u201cEnele distribuzione\u201d
i Carabinieri del Comando Stazione di Fioran

In [14]:
db = Neo4jDb("bolt://localhost:7687", "neo4j", "neo4j_password")
db.update_data()
db.close()